In [1]:
import numpy as np
from numpy.linalg import inv
import sympy as sp
import pandas as pd
import math as m
from sympy import collect, simplify, expand, fraction, latex, diff, cancel, nsimplify
from IPython.display import display, Markdown, Math
from scipy.integrate import odeint
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]

In [2]:
theta, theta_d, I, E_bar, w, w_dot = sp.symbols("\\theta \\theta_d I \\bar{E} \omega \dot{\omega}")
e_theta, e_w, e_I = sp.symbols("e_{\\theta} e_{\omega} e_{I} ")
# e_theta_k, e_w_k, e_I_k = sp.symbols("e_{\\theta_{k}} e_{\omega_{k}} e_{I_{k}}")
e_theta_dot, e_omega_dot, e_I_dot = sp.symbols("\dot{e_{\\theta}} \dot{e_{\omega}} \dot{e_{I}}")
J, D, M, g, L, N1, N2, Kt = sp.symbols("J D M g L N_1 N_2 K_t") 
La, Ra, Kb, u = sp.symbols("L_a R_a K_b u")
Ja, Da, DL = sp.symbols("J_a D_a D_L")
delta = sp.Symbol("\Delta")

# Problem 1

## part 1

In [10]:
M_val = 5
L_val = 1
Ra_val = 10
La_val = 0.1
g_val = 9.8
Kt_val = 11
Kb_val = 6.36
DL_val = 1
Da_val = 0.1
Ja_val = 2
N1_val = 15
N2_val = 45
theta_d_val = np.pi/6
Ts = 0.01

J_val = Ja_val + (N1_val/N2_val)**2*M_val*L_val**2
D_val = Da_val + (N1_val/N2_val)**2*DL_val
E_bar = Ra_val*g_val*N1_val/(Kt_val*L_val*N2_val)*np.sin(N1_val/N2_val*theta_d_val)

lst_subs = [(M,M_val), (L,L_val), (Ra,Ra_val), (La,La_val), (g,g_val), (Kt,Kt_val), (Kb,Kb_val), 
            (D,D_val), (J,J_val), (N1,N1_val), (N2,N2_val), 
            (theta_d, theta_d_val), (delta, Ts)]

# eq 3a
org_3a = w
dot_e_theta = e_w
e_theta_k_1 = e_theta + delta*dot_e_theta
e_theta_k_1_subd = e_theta_k_1.subs(lst_subs)
e_theta_k_1_func = sp.lambdify([e_theta, e_w], e_theta_k_1_subd)

# eq 3b
org_3b = sp.Eq(J*w_dot + D*w + M*g*L*N1/N2*(sp.sin(N1/N2*(e_theta + theta_d)) - sp.sin(N1/N2*theta_d)), Kt*e_I)
dot_e_w = sp.solve(org_3b, w_dot)[0].subs(w, e_w)
e_w_k_1 = e_w + delta*dot_e_w
e_w_k_1_subd = e_w_k_1.subs(lst_subs)
e_w_k_1_func = sp.lambdify([e_theta, e_w, e_I, D, J], e_w_k_1_subd)

# eq 3c
org_3c = sp.Eq(La*e_I_dot + Ra*e_I + Kb*w, u)
dot_e_I = sp.solve(org_3c, e_I_dot)[0].subs(w, e_w)
e_I_k_1 = e_I+ delta*dot_e_I
e_I_k_1_subd = e_I_k_1.subs(lst_subs)
e_I_k_1_func = sp.lambdify([e_w, u], e_I_k_1_subd)
e_I_k_1_coeffs = sp.Poly(e_I_k_1, u).coeffs()

# g_xk = sp.Matrix([[0], [0], [e_I_k_1_coeffs[0]*u]])
g_xk = sp.Matrix([[0], [0], [e_I_k_1_coeffs[0]]])
f_xk = sp.Matrix([[e_theta_k_1], [e_w_k_1], [e_I_k_1_coeffs[1]]])

# u is used as a temporary var. makes turning into np array easier
g_xk_func = sp.lambdify([u], g_xk.subs(lst_subs)*u) 
g_xk_subd = g_xk_func(1)

f_xk_func = sp.lambdify([e_theta, e_w, e_I], f_xk.subs(lst_subs))


Matrix([
[                                                                                                           0.01*e_{\omega} + e_{\theta}],
[0.0430434782608696*e_{I} + 0.999173913043478*e_{\omega} - 0.0639130434782609*sin(e_{\theta}/3 + 0.174532925199433) + 0.0110983835291473],
[                                                                                                                      -0.636*e_{\omega}]])

array([[ 1.02      ],
       [ 2.10749484],
       [-1.272     ]])

In [4]:
u

u

$e_{\theta_{k+1}} = \;${{e_theta_k_1}}

$e_{\omega_{k+1}} = \;${{e_w_k_1}}

$e_{I_{k+1}} = \;${{e_I_k_1}}

$f(\cdot) =\;$ {{f_xk}}

$g(\cdot) =\;$ {{g_xk}}

## part 2

In [34]:
NL = 9
gamma = 0.9
alpha = 2

theta0 = 0
w0 = 0
I0 = 0

e_theta0 = theta0 - theta_d_val
e_w0 = w0
e_I0 = I0 - E_bar/Ra_val

R = 0.05
q = np.array([[1, 0], 
            [0, 1]])

W0 = np.random.randint(0,10,(1, NL)).T
W = [W0]

xk = [np.array([e_theta0, e_w0, e_I0]).reshape(-1,1)]


phi = sp.Matrix([[e_theta, e_w, e_I, 
                 e_theta*e_w, e_theta*e_I, e_w*e_I, 
                 e_theta**2, e_w**2, e_I**2]])

del_phi = sp.Matrix([[diff(phi, e_theta)], [diff(phi, e_w)], [diff(phi, e_I)]])

phi_func = sp.lambdify([e_theta, e_w, e_I], phi)
del_phi_func = sp.lambdify([e_theta, e_w, e_I], del_phi)
# phi = np.array([]).reshape(-1,1)

lst_u = [float((-gamma/(2*R)*g_xk_subd.T@del_phi_func(*xk[-1].T.tolist()[0])@W[-1]).reshape(()))]

[-4.237265971630289]

In [37]:
stop_time = 20 # in seconds
cycles = int(stop_time//Ts)

for k in range(cycles):
    xk.append(f_xk_func(*xk[-1].T.tolist()[0]) + g_xk_subd*lst_u[-1])

In [40]:
len(xk)

2000

In [30]:
f_xk_func(*xk[-1].tolist()[0])

array([[-0.52359878],
       [ 0.00887871],
       [-0.        ]])